In [1]:
import math
import torch
import gpytorch
import numpy as np
import numpy.linalg as linalg

import spectralgp

from spectralgp.samplers import AlternatingSampler
from spectralgp.models import ExactGPModel, SpectralModel, ProductKernelSpectralModel

from spectralgp.sampling_factories import ss_factory, ess_factory
from custom_plotting import plot_kernel

import data

import utils
import argparse

import sys
import matplotlib.pyplot as plt

import traceback

torch.set_default_dtype(torch.float64)

In [2]:
train_x, train_y, test_x, test_y, y_std, y_std_train, gen_kern = data.read_data('servo', nx=None, gen_pars=None,
                                                            linear_pars=None,
                                                            spacing='random',
                                                            noise=None)
in_dims = 1 if train_x.dim() == 1 else train_x.size(1)

use_cuda = torch.cuda.is_available()
print('Cuda is available', use_cuda)
if use_cuda:
    torch.set_default_tensor_type(torch.cuda.DoubleTensor)
    train_x, train_y, test_x, test_y, y_std = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda(), y_std.cuda()
    if gen_kern is not None:
        gen_kern = gen_kern.cuda()

###########################################
## set up the spectral and latent models ##
###########################################
print("Input Dimensions {}".format(in_dims))



mlatent = 'shared'
shared = True if mlatent == 'shared' else False

Cuda is available True
Input Dimensions 4


In [3]:
data_lh = gpytorch.likelihoods.GaussianLikelihood(noise_prior=gpytorch.priors.SmoothedBoxPrior(1e-8, 1e-3))
data_mod = spectralgp.models.ProductKernelSpectralModel(train_x, train_y, data_lh, shared=shared,
        normalize = False, symmetrize = False, num_locs = 100, spacing='random', pretrain=False, omega_max = 8., nonstat = True)

0
tensor(8.)
0
tensor(8.)
0
tensor(8.)


In [4]:
alt_sampler = spectralgp.samplers.AlternatingSampler(
    [data_mod], [data_lh], 
    spectralgp.sampling_factories.ss_factory, [spectralgp.sampling_factories.ess_factory],
    totalSamples=10, numInnerSamples=10, numOuterSamples=5, num_dims=in_dims
    )


alt_sampler.run()

Step:  0 Dimension:  0
Loss is:  tensor(-84.4658, grad_fn=<DivBackward0>)
Loss is:  tensor(-83.2822, grad_fn=<DivBackward0>)
Loss is:  tensor(-82.1321, grad_fn=<DivBackward0>)
Loss is:  tensor(-80.9971, grad_fn=<DivBackward0>)
Loss is:  tensor(-79.9002, grad_fn=<DivBackward0>)
Task: 0 ; Iteration 0
Step:  0 Dimension:  1
Loss is:  tensor(-79.4123, grad_fn=<DivBackward0>)
Loss is:  tensor(-78.3284, grad_fn=<DivBackward0>)
Loss is:  tensor(-77.2648, grad_fn=<DivBackward0>)
Loss is:  tensor(-76.2081, grad_fn=<DivBackward0>)
Loss is:  tensor(-75.1706, grad_fn=<DivBackward0>)
Task: 0 ; Iteration 0
Step:  0 Dimension:  2
Loss is:  tensor(-74.0962, grad_fn=<DivBackward0>)
Loss is:  tensor(-73.0765, grad_fn=<DivBackward0>)
Loss is:  tensor(-72.0711, grad_fn=<DivBackward0>)
Loss is:  tensor(-71.0838, grad_fn=<DivBackward0>)
Loss is:  tensor(-70.1122, grad_fn=<DivBackward0>)
Task: 0 ; Iteration 0
Step:  0 Dimension:  3
Loss is:  tensor(-69.1951, grad_fn=<DivBackward0>)
Loss is:  tensor(-68.2495,

Loss is:  tensor(-13.6643, grad_fn=<DivBackward0>)
Loss is:  tensor(-13.5082, grad_fn=<DivBackward0>)
Task: 0 ; Iteration 6
Step:  6 Dimension:  3
Loss is:  tensor(-13.2398, grad_fn=<DivBackward0>)
Loss is:  tensor(-13.0606, grad_fn=<DivBackward0>)
Loss is:  tensor(-12.9192, grad_fn=<DivBackward0>)
Loss is:  tensor(-12.7689, grad_fn=<DivBackward0>)
Loss is:  tensor(-12.6050, grad_fn=<DivBackward0>)
Task: 0 ; Iteration 6
Seconds for Iteration 6 : 4.846317768096924
Step:  7 Dimension:  0
Loss is:  tensor(-12.3334, grad_fn=<DivBackward0>)
Loss is:  tensor(-12.1776, grad_fn=<DivBackward0>)
Loss is:  tensor(-12.0223, grad_fn=<DivBackward0>)
Loss is:  tensor(-11.8702, grad_fn=<DivBackward0>)
Loss is:  tensor(-11.7462, grad_fn=<DivBackward0>)
Task: 0 ; Iteration 7
Step:  7 Dimension:  1
Loss is:  tensor(-11.8561, grad_fn=<DivBackward0>)
Loss is:  tensor(-11.6807, grad_fn=<DivBackward0>)
Loss is:  tensor(-11.5546, grad_fn=<DivBackward0>)
Loss is:  tensor(-11.4129, grad_fn=<DivBackward0>)
Loss 

In [5]:
data_mod.eval()
data_lh.eval()

data_mod_means = torch.zeros_like(data_mod(test_x).mean)
total_variance = torch.zeros_like(data_lh(data_mod(test_x)).variance)

test_rmse = 0.0
unnorm_test_rmse = 0.0
nll_sum = 0.0
msll = 0.0

with torch.no_grad():
    marg_samples_num = min(len(alt_sampler.fhsampled[0][0]), alt_sampler.fgsampled[0].shape[-1])
    
    for x in range(0, marg_samples_num):
        # This line must come first
        data_mod.load_state_dict(alt_sampler.fhsampled[0][0][x]) # dim, ???, nsample
        
        for dim in range(0,in_dims):
            data_mod.covar_module.set_latent_params(alt_sampler.fgsampled[dim][0, :, x], idx=dim)
            
        # Verify this is the correct way to handle multidim train setting

        data_mod.set_train_data(train_x, train_y) # to clear out the cache
        data_mod_means += data_mod(test_x).mean

        y_preds = data_lh(data_mod(test_x))
        # y_var = f_var + data_noise
        y_var = y_preds.variance
        total_variance += (y_var + torch.pow(data_mod(test_x).mean,2))


meaned_data_mod_means = data_mod_means / float(marg_samples_num)
total_variance = total_variance/float(marg_samples_num) - torch.pow(meaned_data_mod_means,2)

d = meaned_data_mod_means - test_y
du = d * y_std

test_rmse = torch.sqrt(torch.mean(torch.pow(d, 2)))
unnorm_test_rmse = torch.sqrt(torch.mean(torch.pow(du, 2)))

nll = 0.5 * torch.log(2. * math.pi * total_variance) +  torch.pow((meaned_data_mod_means - test_y),2)/(2. * total_variance)
sll = nll - (0.5 * torch.log(2. * math.pi * torch.pow(y_std_train, 2)) +  torch.pow((torch.mean(train_y) - test_y),2)/(2. * torch.pow(y_std_train, 2)))
msll += torch.mean(sll)
nll_sum += nll.sum()

print("Normalised RMSE: {}".format(test_rmse))
print("Unnormalised RMSE: {}".format(unnorm_test_rmse))
print("Summed NLL: {}".format(nll_sum))
print("MSLL: {}".format(msll))

Normalised RMSE: 0.20234513950116603
Unnormalised RMSE: 0.18199989698079141
Summed NLL: 3.8151302280576425
MSLL: -1.3795751601879567


In [6]:
plot_kernel(alt_sampler, data_mod, 'servo')

/home/jayson_s/anaconda3/envs/spectralgp/lib/python3.7/site-packages/matplotlib-3.0.3-py3.7-linux-x86_64.egg/matplotlib/axes/_base.py:380: MatplotlibDeprecationWarning: 
cycling among columns of inputs with non-matching shapes is deprecated.
  cbook.warn_deprecated("2.2", "cycling among columns of inputs "
